In [ ]:
import valurap.commands as vac
import valurap.printer as vp
import valurap.axes as vax
import valurap.apgs as vap
import valurap.asg as vas
import valurap.spi as vs

import valurap.path_planning as vapp

from importlib import reload
reload(vs)
reload(vac)
reload(vas)
reload(vax)
reload(vap)
reload(vp)
reload(vapp)

prn = vp.Valurap()
prn.setup()
prn.axe_e2.max_v=20000*3
prn.axe_e1.max_v=20000*22

In [ ]:
prn.home()

In [ ]:
prn.update_axes_positions()

prn.axe_e1.enabled = True
prn.axe_x1.enabled = True
prn.axe_y.enabled = True
prn.axe_z.enabled = True
prn.update_axes_config()

prn.moveto(X1=0)
prn.moveto(Y=0)

In [ ]:
#prn.moveto(X1=0)
#prn.moveto(Y=0)

#prn.move(Z=-1600*30)
x1, y1 = [255.1652, 288.87604]
x2, y2 = [339.1803, 288.58218]

x3, y3 = [339.1803, 288.58218]
x4, y4 = [339.62244, 372.909]

x, y = [358.5733, 240.30933]


dx = (x - 320)/(x2 - x1)*10
dy = (y - 240)/(y4 - y3)*10
display(dx, dy)
z1 = [
    [320.05545, 240.03896],
    [320.58078, 243.7491],
    [321.13934, 247.78864],
    [321.85776, 252.33138]
]

dx = -40
dy = 0
prn.move(X2=-80*dx, Y=-80*dy)
#prn.move(Z=-1600*30)
#prn.move(X2=-80*10)


In [ ]:
if 0:
    prn.axe_e1.enabled = True
    prn.axe_x1.enabled = True
    prn.axe_y.enabled = True
    prn.axe_z.enabled = True
    prn.axe_e2.enabled = False
    prn.update_axes_config()

#prn.move(E1=-100000)
#prn.move(E1=-20000)
#prn.moveto(X1=1000)
#prn.moveto(X2=0)
#prn.moveto(Y=-3000)
prn.moveto(Z=5040)
prn.moveto(Z=5040-1600*(0.7-0.3))

In [ ]:
prn.axe_e2.max_v=20000*3
prn.axe_e1.max_v=20000*22
display(22/3)
display(3*3/1.75/1.75)

In [ ]:
display(prn.axe_x1.last_pos)
display(prn.axe_x2.last_pos)
display(prn.axe_y.last_pos)

In [ ]:
# open-loop
import valurap.asg as vas
from valurap import path_planning, emulate
import pickle
import imp

imp.reload(vas)
imp.reload(path_planning)

spm = 80
spme = 847
acc_step = 10000

apgs = {
    "X": prn.apg_x,
    "Y": prn.apg_y,
    "Z": prn.apg_z,
}

codes = []

up_segs = path_planning.ext_to_code(0.3, 100, spm=1600, max_a=1000)
up_code = prn.asg.gen_path_code(up_segs, accel_step=int(50000000/acc_step), real_apgs=apgs)

for axe in prn.axes.values():
    axe.apg = None
    axe.enabled = False

prn.axe_z.enabled = True
prn.axe_y.enabled = True
prn.axe_x1.enabled = True
prn.axe_e1.enabled = True
prn.update_axes_config()

start = None
apg_map = None

for layer in range(1, 3000):
    with open("test22_{:05d}.layer".format(layer), "rb") as f:
        p = pickle.load(f)

    if len(p) == 1 and p[0][0] == "do_home":
        break
        
    assert len(p) == 2
    assert p[0][0] == "start"
    assert p[1][0] == "segment"
    if start is None:
        start = p[0]

    pr_opt = [
        [1, [
            vas.ProfileSegment(apgs["X"], x=p[0][1]["X"] * spm),
            vas.ProfileSegment(apgs["Y"], x=p[0][1]["Y"] * spm),
            vas.ProfileSegment(apgs["Z"], x=0),
        ]]
    ]
    cms, meta, segments = p[1]
    if apg_map is None:
        apg_map = meta["map"]
        
    codes.append(prn.asg.gen_map_code(meta["map"]))
    for dt, segs in segments:
        pr_opt.append([
            dt, [vas.ProfileSegment.from_tuple(s, apgs) for s in segs]
        ])

    acc_step = 10000
    path_code = prn.asg.gen_path_code(pr_opt, accel_step=int(50000000/acc_step), real_apgs={"X": prn.apg_x, "Y": prn.apg_y, "Z": prn.apg_z})
    print(len(path_code))
    codes.append(path_code)
    codes.append(prn.asg.gen_map_code({"Z": "Z"}))
    codes.append(up_code)

prn.update_axes_config()
fullcode = []
for code in codes:
    fullcode.extend(code[:-1])
fullcode.extend(code[-1:])

print(len(fullcode))

prn.setup()

for axe in prn.axes.values():
    axe.apg = None
    axe.enabled = False
    
prn.update_axes_config()
prn.home()

prn.update_axes_positions()

prn.axe_e1.enabled = True
prn.axe_x1.enabled = True
prn.axe_y.enabled = True
prn.axe_z.enabled = True
prn.axe_e2.enabled = False
prn.update_axes_config()

prn.move(E1=-100000)
#prn.move(E1=-20000)
prn.moveto(X1=0)
prn.moveto(Y=0)
#prn.moveto(X2=0)
prn.moveto(Y=start[1]["Y"] * 80, X1=start[1]["X"] * 80)
prn.moveto(Z=5040)
#prn.moveto(Z=5040-1600*1.5) # clear glass
prn.moveto(Z=5040-1600*(0.6-0.3))

for k, v in apg_map.items():
    prn.axes[k].apg = prn.apgs[v]
    
#print(start[2])
#prn.moveto(X1=start[1]["X"] * 80, Y=start[1]["Y"] * 80)

try:
    prn.exec_long_code(fullcode, splits=2000, verbose=True)
    
    for axe in prn.axes.values():
        axe.apg = None
        axe.enabled = False
    
    prn.update_axes_config()
    prn.home()
    prn.setup()
except:
    prn.setup()
    


In [ ]:
# check

import valurap.asg as vas
from valurap import path_planning, emulate
import pickle
import imp

imp.reload(vas)
imp.reload(path_planning)
imp.reload(emulate)

spm = 80
spme = 847
acc_step = 10000

apgs = {
    "X": prn.apg_x,
    "Y": prn.apg_y,
    "Z": prn.apg_z,
}

codes = []

up_segs = path_planning.ext_to_code(0.3, 100, spm=1600, max_a=1000)
up_code = prn.asg.gen_path_code(up_segs, accel_step=int(50000000/acc_step), real_apgs={"X": prn.apg_x, "Y": prn.apg_y, "Z": prn.apg_z})

for axe in prn.axes.values():
    axe.apg = None
    axe.enabled = False

if 1:
    prn.axe_z.enabled = True
    prn.axe_y.enabled = True
    prn.axe_x1.enabled = True
    prn.axe_e1.enabled = True

apg_states = {}
codes = []
first = True
for layer in range(1, 100):
    with open("test5_{:05d}.layer".format(layer), "rb") as f:
        p = pickle.load(f)

    assert len(p) == 2
    assert p[0][0] == "start"
    assert p[1][0] == "segment"
    
    print(p[0])
    expected_pos = p[0][1]

    if apg_states:
        print(expected_pos["X"] - apg_states["X"].x / 2**32 / spm)
        print(expected_pos["Y"] - apg_states["Y"].x / 2**32 / spm)

    cms, meta, segments = p[1]
    code = prn.asg.gen_map_code(meta["map"])[:-1]

    pr_opt = []
    
    if first:
        pr_opt += [
            [1, [
                vas.ProfileSegment(apgs["X"], x=int(p[0][1]["X"] * spm)),
                vas.ProfileSegment(apgs["Y"], x=int(p[0][1]["Y"] * spm)),
                vas.ProfileSegment(apgs["Z"], x=0),
            ]]
        ]
        first = False

    
    for dt, segs in segments:
        pr_opt.append([
            dt, [vas.ProfileSegment.from_tuple(s, apgs) for s in segs]
        ])

    if 0:
        pr_opt += [
            [1, [
                vas.ProfileSegment(apgs["X"], v=0),
                vas.ProfileSegment(apgs["Y"], v=0),
                vas.ProfileSegment(apgs["Z"], v=0),
            ]]
        ]

    acc_step = 10000
    path_code = prn.asg.gen_path_code(pr_opt, accel_step=int(50000000/acc_step), real_apgs={"X": prn.apg_x, "Y": prn.apg_y, "Z": prn.apg_z})
    code += path_code[:-1]
    print(len(path_code))
    
    emulate.emulate(pr_opt, accel_step=50000000/acc_step, verbose=0, apg_states=apg_states, no_tracking=True)
    emu_x = int(apg_states["X"].x) & 0xffffffffff000000
    emu_y = int(apg_states["Y"].x) & 0xffffffffff000000
    
    code += prn.asg.gen_map_code({"Z": "Z"})[:-1]
    code += up_code
    codes.append([expected_pos, (emu_x, emu_y), code])

prn.update_axes_config()

deltas = []
try:
    for exp_pos, (exp_x, exp_y), code in codes:
        prn.exec_long_code(code, splits=2000, verbose=True)
        
        cur_x_hi = prn.s3g.S3G_INPUT(prn.apg_x.cur_x_hi)
        cur_y_hi = prn.s3g.S3G_INPUT(prn.apg_y.cur_x_hi)
        cur_x_lo = prn.s3g.S3G_INPUT(prn.apg_x.cur_x_lo)
        cur_y_lo = prn.s3g.S3G_INPUT(prn.apg_y.cur_x_lo)

        if cur_x_hi < 0:
            cur_x_hi += 2**32

        if cur_y_hi < 0:
            cur_y_hi += 2**32

        if cur_x_lo < 0:
            cur_x_lo += 2**32

        if cur_y_lo < 0:
            cur_y_lo += 2**32
            
        cur_x = cur_x_hi << 32 | cur_x_lo
        cur_y = cur_y_hi << 32 | cur_y_lo

        
        if 0:
            if exp_x > 2 ** 63:
                exp_x -= 2 ** 63

            if exp_y > 2 ** 63:
                exp_y -= 2 ** 63

            if cur_x > 2 ** 63:
                cur_x -= 2 ** 63

            if cur_y > 2 ** 63:
                cur_y -= 2 ** 63

            exp_x /= 2 ** 32 * spm
            exp_y /= 2 ** 32 * spm
            cur_x /= 2 ** 32 * spm
            cur_y /= 2 ** 32 * spm

        print("AAAAAAAA", exp_x, exp_y)
        print("AAAAAAAA", cur_x, cur_y)
        deltas.append([exp_x, cur_x, exp_y, cur_y])

except:
    prn.setup()
    print(deltas)
    raise

    
print(deltas)

In [ ]:
for axe in prn.axes.values():
    axe.apg = None
    axe.enabled = False
    
prn.update_axes_config()
prn.home()

In [ ]:
prn.moveto(X1=-13600)

In [ ]:
from valurap import emulate

apgs = {
    "X": prn.apg_x,
    "Y": prn.apg_y,
    "Z": prn.apg_z,
}

for axe in prn.axes.values():
    axe.apg = None
    axe.enabled = False

if 0:
    prn.axe_z.enabled = True
    prn.axe_y.enabled = True
    prn.axe_x1.enabled = True
    prn.axe_e1.enabled = True
    
pr_opt = []
    
pr_opt += [
    [1, [
        vas.ProfileSegment(apgs["X"], x=0),
        vas.ProfileSegment(apgs["Y"], x=0),
        vas.ProfileSegment(apgs["Z"], x=0),
    ]],
    [1000, [
        vas.ProfileSegment(apgs["X"], v=-1000000, a=1000000, j=-10000),
        vas.ProfileSegment(apgs["Y"], v=-1000000, a=1000000, j=-10001),
        vas.ProfileSegment(apgs["Z"], v=0),
    ]],
    [1000, [
        vas.ProfileSegment(apgs["X"], a=-1000000, j=11000),
        vas.ProfileSegment(apgs["Y"], a=-1000000, j=11001),
        vas.ProfileSegment(apgs["Z"], v=0),
    ]],
    [1, [
        vas.ProfileSegment(apgs["X"], v=0),
        vas.ProfileSegment(apgs["Y"], v=0),
        vas.ProfileSegment(apgs["Z"], v=0),
    ]],
]

acc_step = 10000
code = prn.asg.gen_path_code(pr_opt, accel_step=int(50000000/acc_step), real_apgs=apgs)
prn.exec_long_code(code, splits=2000, verbose=True)
cur_x = prn.s3g.S3G_INPUT(prn.apg_x.cur_x_hi)
cur_y = prn.s3g.S3G_INPUT(prn.apg_y.cur_x_hi)
cur_x_lo = prn.s3g.S3G_INPUT(prn.apg_x.cur_x_lo)
cur_y_lo = prn.s3g.S3G_INPUT(prn.apg_y.cur_x_lo)

if cur_x < 0:
    cur_x += 2**32

if cur_y < 0:
    cur_y += 2**32

if cur_x_lo < 0:
    cur_x_lo += 2**32

if cur_y_lo < 0:
    cur_y_lo += 2**32

apg_states = {}
res = emulate.emulate(pr_opt, accel_step=50000000/acc_step, verbose=0, apg_states=apg_states)

print(cur_x << 32 | cur_x_lo)
print(cur_y << 32 | cur_y_lo)
print(int(apg_states["X"].x) & 0xffffffffff000000)
print(int(apg_states["Y"].x) & 0xffffffffff000000)